In [2]:
from itertools import chain

In [3]:
#Sriramajayam
import re
training_file = open("test_document.txt",encoding="utf-8")
training_data = training_file.read()
training_data_doc_contents_list = re.findall(r"ID tr_doc_[0-9]+[\n]TEXT(.*?)[\n]EOD", training_data)
print(training_data_doc_contents_list[0])

 This is file number 1.


In [4]:
# testing_file = open("testing_docs.txt",encoding="utf-8")
# testing_data = testing_file.read()
# testing_data_doc_contents_list = re.findall(r"ID te_doc_[0-9]+[\n]TEXT(.*?)[\n]EOD", testing_data)
# print(testing_data_doc_contents_list[0])

In [5]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# # List of list for training data features' list
# training_data_feature_list=[]
# for each in training_data_doc_contents_list:
#     vectorizer = TfidfVectorizer(stopwords='english')
#     content_list = []
#     if(len(each)>2):
#         X = vectorizer.fit_transform([each])
#         content_list = vectorizer.get_feature_names()
#     training_data_feature_list.append(content_list)



In [6]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# # List of list for testing data features' list
# testing_data_feature_list =[]
# for each in testing_data_doc_contents_list:
#     vectorizer = TfidfVectorizer(stop_words='english')
#     content_list = []
#     if(len(each)>2):
#         X = vectorizer.fit_transform(C1_list)
#         content_list = vectorizer.get_feature_names()
#     testing_data_feature_list.append(content_list)

# print(testing_data_feature_list[0])

In [7]:
import pandas as pd
training_data_df = pd.read_csv("test_labels.txt",sep=' ',names=['doc_id','topic_id'])
training_data_df.head()


,doc_id,topic_id
0,tr_doc_00001,C1
1,tr_doc_00002,C1
2,tr_doc_00003,C1
3,tr_doc_00004,C2


In [8]:
training_data_df['content'] = pd.Series(training_data_doc_contents_list)

In [9]:
training_data_df.head()

,doc_id,topic_id,content
0,tr_doc_00001,C1,This is file number 1.
1,tr_doc_00002,C1,This is file number 2 file.
2,tr_doc_00003,C1,This is file number 3.
3,tr_doc_00004,C2,This is file number 4.


In [10]:
import nltk
def stemmer(doc):
    stemmer = nltk.stem.SnowballStemmer('english')
    stemmed_doc = stemmer.stem(doc)
    return stemmed_doc   

In [11]:
import nltk
from sklearn.feature_extraction.text import CountVectorizer

stemmer = nltk.stem.SnowballStemmer('english')
class StemmedTfidfVectorizer(CountVectorizer):

    def build_analyzer(self):
        analyzer = super(CountVectorizer, self).build_analyzer()
        return lambda doc: (stemmer.stem(w) for w in analyzer(doc))

In [12]:
from itertools import chain
import itertools
topic_groups = training_data_df.groupby('topic_id')
vectorizer = StemmedTfidfVectorizer(stop_words='english', ngram_range=(1,3),analyzer='word')
#save_group_content = open("29406242_group.txt", 'a')
file_index = 1

for group_name, each_topic in topic_groups:  
    
    if group_name == 'C1':
        x = vectorizer.fit_transform(each_topic['content'])
        mapping_dict = vectorizer.vocabulary_

        coordinates = x.tocoo()
        save_term_doc_matrix_name = "29406242_tfidf_"+ str(file_index) + ".txt"
        save_term_doc_matrix = open(save_term_doc_matrix_name, 'w')

        for i, j, v in itertools.zip_longest(coordinates.row, coordinates.col, coordinates.data):
            save_term_doc_matrix.write(str(i) + ':' + str(j) + ':' + str(v)+ "\n")
        save_term_doc_matrix.write("***************************************************************")
        save_term_doc_matrix.close()

        save_term_index_map_name = "29406242_vocab_"+ str(file_index) + ".txt"
        save_term_index_map = open(save_term_index_map_name, 'w',encoding='utf-8')
        file_index += 1
        for term, index in sorted(mapping_dict.items()):
            save_term_index_map.write(term + ": " + str(index) + "\n")
        save_term_index_map.write("***************************************************************")
        save_term_index_map.close()       

        print("group_name == ",group_name)  

group_name ==  C1
